<a href="https://colab.research.google.com/github/yy0750/KOSPI200-Components-Direction-Prediction/blob/main/NextDay_240%2C1_Comp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pwd

/content


In [ ]:
%cd drive/MyDrive/code/통계학과머신러닝PBL

/content/drive/MyDrive/code/통계학과머신러닝PBL


In [ ]:
import pandas as pd
import numpy as np
import random
import time
import pickle
from sklearn.preprocessing import OneHotEncoder
from Statistics import Statistics
from sklearn.ensemble import RandomForestClassifier

In [ ]:
import os
SEED = 9
os.environ['PYTHONHASHSEED']=str(SEED)
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
# 논문 코드 수정
KS200_df = pd.read_csv('/content/drive/MyDrive/code/통계학과머신러닝PBL/data/KSXconst.csv')

# NaN 값을 0으로 대체
KS200_df = KS200_df.fillna(0)

# KS200_df에서 소수점 제거
KS200_df = KS200_df.astype(int)

# 모든 값을 문자열로 변환하고 6자리로 맞추기
KS200_df = KS200_df.astype(str)
KS200_df = KS200_df.apply(lambda x: x.str.zfill(6))

all_companies = list(KS200_df.dropna().values.flatten())
all_companies = set(all_companies)

In [ ]:
KS200_df.head()

,2004/01,2004/02,2004/03,2004/04,2004/05,2004/06,2004/07,2004/08,2004/09,2004/10,...,2023/03,2023/04,2023/05,2023/06,2023/07,2023/08,2023/09,2023/10,2023/11,2023/12
0,005930,005930,005930,005930,005930,005930,005930,005930,005930,005930,...,005930,005930,005930,005930,005930,005930,005930,005930,005930,005930
1,017670,017670,017670,017670,017670,017670,017670,005490,005490,005490,...,373220,373220,373220,373220,373220,373220,373220,373220,373220,373220
2,060000,060000,060000,060000,060000,060000,005490,017670,017670,017670,...,000660,000660,000660,000660,000660,000660,000660,000660,000660,000660
3,005490,005490,005490,005490,015760,005490,060000,015760,015760,015760,...,207940,207940,207940,207940,207940,207940,207940,207940,207940,207940
4,015760,015760,015760,015760,005490,015760,015760,060000,060000,060000,...,006400,006400,051910,006400,006400,005490,005490,005490,005380,005490


In [ ]:
# 논문 코드 수정
constituents = {}

for col in KS200_df.columns:
    # 열 이름을 '/'로 분할
    col_parts = col.split('/')

    # 역순으로 된 리스트 항목들을 '-'로 연결하여 새로운 키 생성
    new_key = '-'.join(col_parts)

    # 해당 열의 데이터에서 NaN 제거하고 집합으로 변환
    col_values = set(KS200_df[col].dropna())

    # 새로운 키와 값을 constituents 딕셔너리에 추가
    constituents[new_key] = col_values

print(constituents)

{'2004-01': {'007210', '015760', '032710', '017810', '002030', '009270', '002790', '003480', '002000', '003300', '000880', '008060', '001800', '003030', '000640', '014900', '010620', '001830', '042660', '015860', '025930', '000100', '016380', '000830', '001370', '002300', '004020', '005010', '006600', '006360', '060000', '004000', '025560', '000180', '010140', '002990', '010950', '001210', '008930', '011780', '000810', '006800', '051900', '007340', '003160', '000070', '002240', '025860', '000480', '000150', '025000', '047040', '010120', '064960', '001020', '009830', '005490', '066570', '020760', '029460', '016880', '000010', '006200', '010770', '013520', '002860', '004170', '003410', '027840', '003120', '005950', '004800', '005940', '000020', '000700', '005070', '004940', '006840', '005500', '005870', '036460', '009540', '001940', '002270', '049770', '011170', '006660', '007810', '002540', '010060', '006380', '055550', '000210', '006250', '008350', '025850', '002350', '021240', '012600

In [ ]:
constituents_train = {}
for test_year in range(2007,2021):
    months = [str(t)+'-0'+str(m) if m<10 else str(t)+'-'+str(m)
              for t in range(test_year-3,test_year) for m in range(1,13)]
    constituents_train[test_year] = [list(constituents[m]) for m in months]
    constituents_train[test_year] = set([i for sublist in constituents_train[test_year] for i in sublist])


# XGBClassifier


In [ ]:
# from xgboost import XGBClassifier

# def trainer(train_data, test_data):
#     random.seed(SEED)
#     np.random.seed(SEED)
#     train_x, train_y = train_data[:, 2:-2], train_data[:, -1]
#     train_y = train_y.astype('int')
#     print('Started training')
#     clf = XGBClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1)
#     clf.fit(train_x, train_y)
#     print('Completed ', clf.score(train_x, train_y))
#     dates = list(set(test_data[:, 0]))
#     predictions = {}
#     for day in dates:
#         test_d = test_data[test_data[:, 0] == day]
#         test_d = test_d[:, 2:-2]
#         predictions[day] = clf.predict_proba(test_d)[:, 1]
#     return predictions

In [ ]:
# def simulate(test_data,predictions):
#     rets = pd.DataFrame([],columns=['Long','Short'])
#     k = 10
#     for day in sorted(predictions.keys()):
#         preds = predictions[day]
#         test_returns = test_data[test_data[:,0]==day][:,-2]
#         top_preds = predictions[day].argsort()[-k:][::-1]
#         trans_long = test_returns[top_preds]
#         worst_preds = predictions[day].argsort()[:k][::-1]
#         trans_short = -test_returns[worst_preds]
#         rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
#     return rets

# def create_label(df,perc=[0.5,0.5]):
#     perc = [0.]+list(np.cumsum(perc))
#     label = df.iloc[:,1:].pct_change(fill_method=None)[1:].apply(
#         lambda x: pd.qcut(x.rank(method='first'),perc,labels=False), axis=1)
#     return label

# # 논문 코드 수정
# def create_stock_data(df, st):
#     if st not in df.columns:
#         # st가 df의 열 이름에 없는 경우 None 반환
#         return None, None

#     st_data = pd.DataFrame([])
#     st_data['Date'] = list(df['Date'])
#     st_data['Name'] = [st] * len(st_data)
#     for k in list(range(1, 21)) + list(range(40, 241, 20)):
#         st_data['R' + str(k)] = df[st].pct_change(k)
#     st_data['R-future'] = df[st].pct_change().shift(-1)

#     # label 딕셔너리에 st 키가 없는 경우 처리
#     if st in label:
#         st_data['label'] = list(label[st]) + [np.nan]
#     else:
#         st_data['label'] = [np.nan] * len(st_data)

#     st_data['Month'] = list(df['Date'].str[:-3])
#     st_data = st_data.dropna()
#     trade_year = st_data['Month'].str[:4]
#     st_data = st_data.drop(columns=['Month'])
#     st_train_data = st_data[trade_year < str(test_year)]
#     st_test_data = st_data[trade_year == str(test_year)]
#     return np.array(st_train_data), np.array(st_test_data)

In [ ]:
# result_folder = 'results-NextDay-240-1-RF'
# for directory in [result_folder]:
#     if not os.path.exists(directory):
#         os.makedirs(directory)

In [ ]:
# # 논문코드 수정
# for test_year in range(2007,2024):

#     print('-'*40)
#     print(test_year)
#     print('-'*40)

#     filename = '/content/drive/MyDrive/code/통계학과머신러닝PBL/data/Close_'+str(test_year-3)+'.csv'
#     df = pd.read_csv(filename)

#     label = create_label(df)
#     stock_names = sorted(list(constituents[str(test_year-1)+'-12']))
#     train_data,test_data = [],[]

#     start = time.time()
#     for st in stock_names:
#         st_train_data, st_test_data = create_stock_data(df, st)

#         # st_train_data와 st_test_data가 None이 아닌 경우에만 리스트에 추가
#         if st_train_data is not None and st_test_data is not None:
#             train_data.append(st_train_data)
#             test_data.append(st_test_data)


#     # 빈 배열이 없는 경우에만 concatenate 수행
#     if train_data:
#         train_data = np.concatenate([x for x in train_data])
#     else:
#         train_data = np.array([])

#     if test_data:
#         test_data = np.concatenate([x for x in test_data])
#     else:
#         test_data = np.array([])

#     predictions = trainer(train_data,test_data)
#     returns = simulate(test_data,predictions)
#     result = Statistics(returns.sum(axis=1))
#     print('\nAverage returns prior to transaction charges')
#     result.shortreport()

#     with open(result_folder+'/predictions-'+str(test_year)+'.pickle', 'wb') as handle:
#         pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)

#     returns.to_csv(result_folder+'/avg_daily_rets-'+str(test_year)+'.csv')
#     with open(result_folder+"/avg_returns.txt", "a") as myfile:
#         res = '-'*30 + '\n'
#         res += str(test_year) + '\n'
#         res += 'Mean = ' + str(result.mean()) + '\n'
#         res += 'Sharpe = '+str(result.sharpe()) + '\n'
#         res += '-'*30 + '\n'
#         myfile.write(res)

*   RandomForestClassifier
*   XGBClassifier
*   LogisticRegression
*   LGBMClassifier



In [ ]:
def simulate(test_data,predictions):
    rets = pd.DataFrame([],columns=['Long','Short'])
    k = 10
    for day in sorted(predictions.keys()):
        preds = predictions[day]
        test_returns = test_data[test_data[:,0]==day][:,-2]
        top_preds = predictions[day].argsort()[-k:][::-1]
        trans_long = test_returns[top_preds]
        worst_preds = predictions[day].argsort()[:k][::-1]
        trans_short = -test_returns[worst_preds]
        rets.loc[day] = [np.mean(trans_long),np.mean(trans_short)]
    return rets

def create_label(df,perc=[0.5,0.5]):
    perc = [0.]+list(np.cumsum(perc))
    label = df.iloc[:,1:].pct_change(fill_method=None)[1:].apply(
        lambda x: pd.qcut(x.rank(method='first'),perc,labels=False), axis=1)
    return label

# 논문 코드 수정
def create_stock_data(df, st):
    if st not in df.columns:
        # st가 df의 열 이름에 없는 경우 None 반환
        return None, None

    st_data = pd.DataFrame([])
    st_data['Date'] = list(df['Date'])
    st_data['Name'] = [st] * len(st_data)
    for k in list(range(1, 21)) + list(range(40, 241, 20)):
        st_data['R' + str(k)] = df[st].pct_change(k)
    st_data['R-future'] = df[st].pct_change().shift(-1)

    # label 딕셔너리에 st 키가 없는 경우 처리
    if st in label:
        st_data['label'] = list(label[st]) + [np.nan]
    else:
        st_data['label'] = [np.nan] * len(st_data)

    st_data['Month'] = list(df['Date'].str[:-3])
    st_data = st_data.dropna()
    trade_year = st_data['Month'].str[:4]
    st_data = st_data.drop(columns=['Month'])
    st_train_data = st_data[trade_year < str(test_year)]
    st_test_data = st_data[trade_year == str(test_year)]
    return np.array(st_train_data), np.array(st_test_data)

result_folder = 'results-NextDay-240-1-RF'
for directory in [result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier

def trainer(train_data, test_data, model_name):
    random.seed(SEED)
    np.random.seed(SEED)
    train_x, train_y = train_data[:, 2:-2], train_data[:, -1]
    train_y = train_y.astype('int')
    print('Started training')

    if model_name == 'rf':
        clf = RandomForestClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1)
    elif model_name == 'xgb':
        clf = XGBClassifier(n_estimators=1000, max_depth=20, random_state=SEED, n_jobs=-1)
    elif model_name == 'lr':
        clf = LogisticRegression(random_state=SEED, n_jobs=-1)
    elif model_name == 'lightgbm':
        clf = LGBMClassifier(random_state=SEED, n_jobs=-1)
    else:
        raise ValueError('Invalid model name')

    clf.fit(train_x, train_y)
    print('Completed ', clf.score(train_x, train_y))

    dates = list(set(test_data[:, 0]))
    predictions = {}
    for day in dates:
        test_d = test_data[test_data[:, 0] == day]
        test_d = test_d[:, 2:-2]
        predictions[day] = clf.predict_proba(test_d)[:, 1]
    return predictions

result_folder = 'results-NextDay-240-1-All'
for directory in [result_folder]:
    if not os.path.exists(directory):
        os.makedirs(directory)

for test_year in range(2023, 2024):
    print('-' * 40)
    print(test_year)
    print('-' * 40)

    filename = '/content/drive/MyDrive/code/통계학과머신러닝PBL/data/Close_' + str(test_year - 3) + '.csv'
    df = pd.read_csv(filename)

    label = create_label(df)
    stock_names = sorted(list(constituents[str(test_year - 1) + '-12']))
    train_data, test_data = [], []

    start = time.time()
    for st in stock_names:
        st_train_data, st_test_data = create_stock_data(df, st)

        # st_train_data와 st_test_data가 None이 아닌 경우에만 리스트에 추가
        if st_train_data is not None and st_test_data is not None:
            train_data.append(st_train_data)
            test_data.append(st_test_data)

    # 빈 배열이 없는 경우에만 concatenate 수행
    if train_data:
        train_data = np.concatenate([x for x in train_data])
    else:
        train_data = np.array([])

    if test_data:
        test_data = np.concatenate([x for x in test_data])
    else:
        test_data = np.array([])

    # 여러 모델에 대한 결과 저장
    model_results = {}

    for model_name in ['rf', 'xgb', 'lr', 'lightgbm']:
        print(f'Running {model_name} model...')
        predictions = trainer(train_data, test_data, model_name)
        returns = simulate(test_data, predictions)
        result = Statistics(returns.sum(axis=1))

        model_results[model_name] = {
            'mean': result.mean(),
            'std_dev': result.std(),
            'sharpe_ratio': result.sharpe()
        }

        print(f'Average returns prior to transaction charges for {model_name} model')
        result.shortreport()

    for test_year in range(2023, 2024):
        # 모델 결과 비교
        model_comparison = pd.DataFrame.from_dict(model_results, orient='index')
        print('\nModel Comparison for ', test_year, ':')
        print(model_comparison)

        with open(result_folder + '/predictions-' + str(test_year) + '.pickle', 'wb') as handle:
            pickle.dump(predictions, handle, protocol=pickle.HIGHEST_PROTOCOL)

        returns.to_csv(result_folder + '/avg_daily_rets-' + str(test_year) + '.csv')
        with open(result_folder + "/avg_returns.txt", "a") as myfile:
            res = '-' * 30 + '\n'
            res += str(test_year) + '\n'
            res += 'Mean = ' + str(result.mean()) + '\n'
            res += 'Sharpe = ' + str(result.sharpe()) + '\n'
            res += '-' * 30 + '\n'
            myfile.write(res)

----------------------------------------
2023
----------------------------------------
Running rf model...
Started training
Completed  0.9783168651296343
Average returns prior to transaction charges for rf model
Mean 		 0.0031374922718313126
Standard dev 	 0.0140763783713971
Sharpe ratio 	 3.1999561393950637
Running xgb model...
Started training
Completed  1.0
Average returns prior to transaction charges for xgb model
Mean 		 0.00048630057216527496
Standard dev 	 0.01090318954577814
Sharpe ratio 	 0.2712444726318928
Running lr model...
Started training
Completed  0.5240372434760577
Average returns prior to transaction charges for lr model
Mean 		 0.003486772337572269
Standard dev 	 0.020251394256802575
Sharpe ratio 	 2.498022699063509
Running lightgbm model...
Started training
[LightGBM] [Info] Number of positive: 47433, number of negative: 47295
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.033060 seconds.
You can set `force_col_wise=true` to 